In [6]:
import json
import h5py
import numpy as np
import cv2
from pathlib import Path
from typing import Dict, List, Set
from collections import defaultdict
import os
from tqdm import tqdm

class ImageExtractor:
    """从JSONL文件读取信息并从HDF5文件中提取对应图像"""
    
    def __init__(self, 
                 base_data_path: str = "/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/benchmark1_0_compressed",
                 output_base_path: str = "/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/images"):
        """
        初始化提取器
        
        Args:
            base_data_path: HDF5数据的基础路径
            output_base_path: 输出图像的基础路径
        """
        self.base_data_path = Path(base_data_path)
        self.output_base_path = Path(output_base_path)
        self.errors = []  # 记录错误
        
    def parse_id(self, id_string: str) -> Dict[str, str]:
        """
        解析ID字符串
        
        Args:
            id_string: 格式如 "h5_franka_3rgb/place_in_block_in_plate_1/1014_163457"
            
        Returns:
            包含解析结果的字典
        """
        parts = id_string.split('/')
        if len(parts) < 3:
            raise ValueError(f"无效的ID格式: {id_string}")
            
        return {
            'embodiment': parts[0],  # h5_franka_3rgb
            'task_name': parts[1],    # place_in_block_in_plate_1
            'episode_id': parts[2]     # 1014_163457
        }
    
    def construct_h5_path(self, id_info: Dict[str, str]) -> Path:
        """
        根据ID信息构建HDF5文件路径
        
        Args:
            id_info: 包含embodiment, task_name, episode_id的字典
            
        Returns:
            HDF5文件的完整路径
        """
        h5_dir = self.base_data_path / id_info['embodiment'] / id_info['task_name'] / \
                 'success_episodes' / 'train' / id_info['episode_id'] / 'data'
        
        # 检查目录是否存在
        if not h5_dir.exists():
            raise FileNotFoundError(f"目录不存在: {h5_dir}")
        
        # 查找HDF5文件（通常名为trajectory.hdf5）
        h5_files = list(h5_dir.glob('*.hdf5'))
        if not h5_files:
            raise FileNotFoundError(f"未找到HDF5文件在: {h5_dir}")
        
        # 优先选择trajectory.hdf5
        for h5_file in h5_files:
            if h5_file.name == 'trajectory.hdf5':
                return h5_file
        
        return h5_files[0]  # 如果没有trajectory.hdf5，返回第一个HDF5文件
    
    def decode_image(self, compressed_image: np.ndarray) -> np.ndarray:
        """
        解码压缩的图像数据
        
        Args:
            compressed_image: 压缩的图像数据
            
        Returns:
            解码后的RGB图像
        """
        if compressed_image is None:
            return None
            
        # 如果是单个图像
        if len(compressed_image.shape) == 1:
            rgb = cv2.imdecode(compressed_image, cv2.IMREAD_COLOR)
            if rgb is None:
                # 尝试直接解析为数组
                rgb = np.frombuffer(compressed_image, dtype=np.uint8)
                # 根据大小判断图像尺寸
                if rgb.size == 2764800:
                    rgb = rgb.reshape(720, 1280, 3)
                elif rgb.size == 921600:
                    rgb = rgb.reshape(480, 640, 3)
            return rgb
        else:
            # 如果是批量图像，返回None（需要特殊处理）
            return None
    
    def extract_specific_frames(self, h5_path: Path, frame_names: List[str]) -> Dict[str, np.ndarray]:
        """
        从HDF5文件中提取特定帧（自动从帧名识别相机）
        
        Args:
            h5_path: HDF5文件路径
            frame_names: 要提取的帧名称列表（如 ["camera_front_0000.jpg", "camera_front_0281.jpg"]）
            
        Returns:
            帧名称到图像数组的映射
        """
        images = {}
        
        with h5py.File(h5_path, 'r') as f:
            # 检查是否压缩
            is_compress = f.attrs.get('compress', True)
            
            # 检查RGB图像路径
            if 'observations' not in f or 'rgb_images' not in f['observations']:
                return images
            
            rgb_images_group = f['observations']['rgb_images']
            
            # 解析帧名并提取图像
            for frame_name in frame_names:
                try:
                    # 从帧名提取相机名称和索引
                    # "camera_front_0000.jpg" -> camera="camera_front", idx=0
                    # "camera_left_wrist_0281.jpg" -> camera="camera_left_wrist", idx=281
                    parts = frame_name.split('_')
                    frame_idx_str = parts[-1].split('.')[0]  # "0000"
                    camera_name = '_'.join(parts[:-1])        # "camera_front"
                    frame_idx = int(frame_idx_str)
                    
                    # 检查相机是否存在
                    if camera_name not in rgb_images_group:
                        continue
                    
                    rgb_data = rgb_images_group[camera_name]
                    
                    # 检查索引是否有效
                    if frame_idx >= len(rgb_data):
                        continue
                    
                    # 提取图像
                    if is_compress:
                        # 解码压缩图像
                        compressed_img = rgb_data[frame_idx]
                        img = self.decode_image(compressed_img)
                        if img is not None:
                            # 转换BGR到RGB
                            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                            images[frame_name] = img
                    else:
                        # 直接读取未压缩图像
                        images[frame_name] = rgb_data[frame_idx]
                        
                except (ValueError, IndexError) as e:
                    continue
        
        return images
    
    def save_images(self, images: Dict[str, np.ndarray], output_dir: Path):
        """
        保存图像到指定目录
        
        Args:
            images: 帧名称到图像数组的映射
            output_dir: 输出目录
        """
        output_dir.mkdir(parents=True, exist_ok=True)
        
        for frame_name, img in images.items():
            output_path = output_dir / frame_name
            # 转换RGB回BGR用于OpenCV保存
            img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            cv2.imwrite(str(output_path), img_bgr)
    
    def process_jsonl(self, jsonl_path: str, limit: int = None):
        """
        处理JSONL文件并提取所有图像
        
        Args:
            jsonl_path: JSONL文件路径
            limit: 限制处理的行数（用于测试）
        """
        # 重置错误列表
        self.errors = []
        
        # 收集所有需要提取的数据
        extraction_tasks = defaultdict(lambda: defaultdict(set))
        
        print(f"📖 读取JSONL文件: {jsonl_path}")
        
        # 读取JSONL文件
        with open(jsonl_path, 'r') as f:
            lines = f.readlines()
            if limit:
                lines = lines[:limit]
                
            for line_num, line in enumerate(lines, 1):
                try:
                    data = json.loads(line.strip())
                    id_string = data['id']
                    
                    # 收集所有需要的帧
                    all_frames = set()
                    
                    # visual_demo中的帧
                    if 'visual_demo' in data:
                        all_frames.update(data['visual_demo'])
                    
                    # stage_to_estimate中的帧
                    if 'stage_to_estimate' in data:
                        all_frames.update(data['stage_to_estimate'])
                    
                    # 按ID和相机分组
                    extraction_tasks[id_string]['frames'].update(all_frames)
                    
                except json.JSONDecodeError:
                    self.errors.append(f"第 {line_num} 行: JSON解析错误")
                except Exception as e:
                    self.errors.append(f"第 {line_num} 行: {str(e)}")
        
        print(f"📊 发现 {len(extraction_tasks)} 个唯一ID需要处理\n")
        
        # 统计成功和失败
        success_count = 0
        failed_count = 0
        
        # 使用tqdm处理每个唯一的ID
        with tqdm(total=len(extraction_tasks), desc="提取图像", unit="ID") as pbar:
            for id_string, task_info in extraction_tasks.items():
                try:
                    # 解析ID
                    id_info = self.parse_id(id_string)
                    
                    # 构建HDF5路径
                    h5_path = self.construct_h5_path(id_info)
                    
                    # 提取所有帧
                    frames_to_extract = list(task_info['frames'])
                    
                    # 从HDF5提取图像
                    images = self.extract_specific_frames(h5_path, frames_to_extract)
                    
                    if images:
                        # 构建输出路径
                        output_dir = self.output_base_path / id_info['embodiment'] / \
                                    id_info['task_name'] / id_info['episode_id']
                        
                        # 保存图像
                        self.save_images(images, output_dir)
                        success_count += 1
                        
                        # 更新进度条描述
                        pbar.set_postfix({
                            '成功': success_count, 
                            '失败': failed_count,
                            '当前': f"{id_info['task_name'][:20]}..."
                        })
                    else:
                        failed_count += 1
                        self.errors.append(f"ID {id_string}: 未能提取任何图像")
                        
                except Exception as e:
                    failed_count += 1
                    self.errors.append(f"ID {id_string}: {str(e)}")
                    pbar.set_postfix({'成功': success_count, '失败': failed_count})
                
                pbar.update(1)
        
        # 输出最终统计
        print(f"\n✅ 处理完成！")
        print(f"   成功: {success_count} 个ID")
        print(f"   失败: {failed_count} 个ID")
        
        # 如果有错误，询问是否查看
        if self.errors:
            print(f"\n⚠️ 发现 {len(self.errors)} 个错误")
            response = input("是否查看错误详情？(y/n): ")
            if response.lower() == 'y':
                print("\n错误列表:")
                for i, error in enumerate(self.errors, 1):
                    print(f"  {i}. {error}")
    
    def diagnose_path(self, id_string: str):
        """
        诊断路径问题，帮助调试
        
        Args:
            id_string: ID字符串
        """
        print(f"\n🔍 诊断ID: {id_string}")
        
        try:
            id_info = self.parse_id(id_string)
            print(f"  解析结果:")
            print(f"    - Embodiment: {id_info['embodiment']}")
            print(f"    - Task: {id_info['task_name']}")
            print(f"    - Episode: {id_info['episode_id']}")
            
            # 检查embodiment目录
            embodiment_path = self.base_data_path / id_info['embodiment']
            print(f"\n  检查embodiment目录: {embodiment_path}")
            if embodiment_path.exists():
                print(f"    ✓ 目录存在")
                
                # 列出所有任务
                task_dirs = sorted([d.name for d in embodiment_path.iterdir() if d.is_dir()])
                print(f"    📁 找到 {len(task_dirs)} 个任务目录")
                
                # 查找相似的任务名
                from difflib import get_close_matches
                similar = get_close_matches(id_info['task_name'], task_dirs, n=3, cutoff=0.6)
                if similar:
                    print(f"    💡 相似的任务名: {similar}")
                
                # 检查确切的任务目录
                task_path = embodiment_path / id_info['task_name']
                if task_path.exists():
                    print(f"\n  ✓ 任务目录存在: {task_path}")
                    
                    # 检查episode路径
                    episode_path = task_path / 'success_episodes' / 'train' / id_info['episode_id']
                    if episode_path.exists():
                        print(f"  ✓ Episode目录存在: {episode_path}")
                        
                        # 检查data目录
                        data_path = episode_path / 'data'
                        if data_path.exists():
                            print(f"  ✓ Data目录存在: {data_path}")
                            
                            # 列出所有文件
                            files = list(data_path.iterdir())
                            print(f"  📁 目录内容:")
                            for f in files:
                                print(f"      - {f.name} ({f.stat().st_size / 1024 / 1024:.2f} MB)")
                            
                            # 查找HDF5文件
                            h5_files = list(data_path.glob('*.hdf5'))
                            if h5_files:
                                print(f"  ✅ 找到HDF5文件: {h5_files[0].name}")
                            else:
                                print(f"  ❌ 未找到HDF5文件")
                        else:
                            print(f"  ❌ Data目录不存在")
                    else:
                        print(f"  ❌ Episode目录不存在")
                        # 列出可用的episodes
                        train_path = task_path / 'success_episodes' / 'train'
                        if train_path.exists():
                            episodes = sorted([d.name for d in train_path.iterdir() if d.is_dir()])[:5]
                            print(f"  💡 可用的episodes示例: {episodes}")
                else:
                    print(f"  ❌ 任务目录不存在: {id_info['task_name']}")
                    print(f"  💡 请检查任务名是否正确")
            else:
                print(f"    ❌ Embodiment目录不存在")
                
        except Exception as e:
            print(f"  ❌ 诊断失败: {e}")
    
    def list_available_tasks(self, embodiment: str = "h5_franka_3rgb"):
        """
        列出指定embodiment下的所有可用任务
        
        Args:
            embodiment: embodiment名称
        """
        embodiment_path = self.base_data_path / embodiment
        
        if not embodiment_path.exists():
            print(f"❌ Embodiment目录不存在: {embodiment_path}")
            return []
        
        tasks = []
        for task_dir in sorted(embodiment_path.iterdir()):
            if task_dir.is_dir():
                # 检查是否有success_episodes/train目录
                train_path = task_dir / 'success_episodes' / 'train'
                if train_path.exists():
                    episodes = list(train_path.iterdir())
                    if episodes:
                        tasks.append({
                            'name': task_dir.name,
                            'episodes': len(episodes)
                        })
        
        print(f"\n📋 {embodiment} 下的可用任务:")
        print(f"{'任务名称':<50} {'Episodes数量':>15}")
        print("-" * 65)
        for task in tasks:
            print(f"{task['name']:<50} {task['episodes']:>15}")
        
        return tasks


# ========== 使用示例 ==========

def main():
    # 创建提取器
    extractor = ImageExtractor(
        base_data_path="/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/benchmark1_0_compressed",
        output_base_path="/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/images"
    )
    
    # 处理JSONL文件
    jsonl_file = "/home/runsheng/personal_3/qiancx/ProgressLM/data/raw/converted/h5_agilex_3rgb_converted.jsonl"
    extractor.process_jsonl(jsonl_file)
    
    # 如果需要诊断特定ID的问题，可以使用：
    # extractor.diagnose_path("h5_agilex_3rgb/10_packplate_2/2024_09_28-16_27_42-172863566445571200.00")
    
    # 列出可用任务：
    # extractor.list_available_tasks("h5_agilex_3rgb")


if __name__ == "__main__":
    main()

📖 读取JSONL文件: /home/runsheng/personal_3/qiancx/ProgressLM/data/raw/converted/h5_agilex_3rgb_converted.jsonl
📊 发现 2262 个唯一ID需要处理



提取图像: 100%|██████████| 2262/2262 [00:52<00:00, 42.85ID/s, 成功=2262, 失败=0, 当前=9_appleyellowplate_2...]



✅ 处理完成！
   成功: 2262 个ID
   失败: 0 个ID


In [ ]:
import h5py
import numpy as np

h5_path = "/home/vcj9002/jianshu/chengxuan/Data/robomind/data/benchmark1_0_compressed/h5_franka_3rgb/close_trash/success_episodes/train/0926_112252/data/trajectory.hdf5"

def explore_h5_structure(path, max_depth=4):
    """递归探索HDF5文件结构"""
    
    def print_structure(name, obj, depth=0):
        if depth > max_depth:
            return
        
        indent = "  " * depth
        
        if isinstance(obj, h5py.Group):
            print(f"{indent}📁 {name}/ (Group)")
            # 打印属性
            if obj.attrs:
                for key, val in obj.attrs.items():
                    print(f"{indent}   @{key} = {val}")
        
        elif isinstance(obj, h5py.Dataset):
            print(f"{indent}📄 {name} (Dataset)")
            print(f"{indent}   Shape: {obj.shape}")
            print(f"{indent}   Dtype: {obj.dtype}")
            if obj.attrs:
                for key, val in obj.attrs.items():
                    print(f"{indent}   @{key} = {val}")
    
    print(f"\n🔍 探索HDF5文件结构: {path}\n")
    print("=" * 80)
    
    with h5py.File(path, 'r') as f:
        # 打印文件级别属性
        print("📋 文件属性:")
        for key, val in f.attrs.items():
            print(f"  @{key} = {val}")
        print()
        
        # 递归打印结构
        f.visititems(print_structure)
        
        # 特别关注rgb_images
        print("\n" + "=" * 80)
        print("🎯 重点检查: RGB图像路径")
        print("=" * 80)
        
        # 尝试常见路径
        paths_to_check = [
            'observations/rgb_images',
            'observations/images',
            'rgb_images',
            'images',
            'obs/rgb_images',
            'obs/images',
        ]
        
        for path_str in paths_to_check:
            if path_str in f:
                print(f"\n✅ 找到: {path_str}")
                obj = f[path_str]
                if isinstance(obj, h5py.Group):
                    print(f"   相机列表: {list(obj.keys())}")
                    for camera in obj.keys():
                        print(f"     - {camera}: shape={obj[camera].shape}, dtype={obj[camera].dtype}")
                else:
                    print(f"   Shape: {obj.shape}, Dtype: {obj.dtype}")
            else:
                print(f"❌ 未找到: {path_str}")

# 运行探索
explore_h5_structure(h5_path)


🔍 探索HDF5文件结构: /home/vcj9002/jianshu/chengxuan/Data/robomind/data/benchmark1_0_compressed/h5_franka_3rgb/place_in_trash/success_episodes/train/0924_144150/data/trajectory.hdf5

📋 文件属性:
  @compress = True
  @sim = False

📄 language_distilbert (Dataset)
   Shape: (1, 1, 768)
   Dtype: float16
📄 language_raw (Dataset)
   Shape: (1,)
   Dtype: object
📁 master/ (Group)
📄 master/joint_position (Dataset)
   Shape: (200, 8)
   Dtype: float64
📁 observations/ (Group)
📁 observations/depth_images/ (Group)
📄 observations/depth_images/camera_left (Dataset)
   Shape: (200,)
   Dtype: object
📄 observations/depth_images/camera_right (Dataset)
   Shape: (200,)
   Dtype: object
📄 observations/depth_images/camera_top (Dataset)
   Shape: (200,)
   Dtype: object
📁 observations/rgb_images/ (Group)
📄 observations/rgb_images/camera_left (Dataset)
   Shape: (200,)
   Dtype: object
📄 observations/rgb_images/camera_right (Dataset)
   Shape: (200,)
   Dtype: object
📄 observations/rgb_images/camera_top (Dataset)
  

In [4]:
import h5py
import numpy as np
import cv2
from pathlib import Path
from typing import Dict, List, Optional
from tqdm import tqdm
import argparse


class RGBImageExtractor:
    """通用RGB图像提取器 - 从HDF5文件批量提取所有RGB图像"""
    
    def __init__(self, 
                 base_data_path: str,
                 output_base_path: str = "/home/vcj9002/jianshu/chengxuan/Data/robomind/data/3rgb"):
        """
        初始化提取器
        
        Args:
            base_data_path: HDF5数据的基础路径
            output_base_path: 输出图像的基础路径（按相机分类）
        """
        self.base_data_path = Path(base_data_path)
        self.output_base_path = Path(output_base_path)
        self.stats = {
            'total_h5_files': 0,
            'successful_h5_files': 0,
            'failed_h5_files': 0,
            'total_images': 0,
            'cameras': {}
        }
        self.errors = []
        
    def decode_image(self, compressed_image: np.ndarray) -> Optional[np.ndarray]:
        """
        解码压缩的图像数据
        
        Args:
            compressed_image: 压缩的图像数据
            
        Returns:
            解码后的RGB图像（已转换为RGB格式）
        """
        if compressed_image is None or len(compressed_image) == 0:
            return None
            
        try:
            # 使用cv2解码
            rgb = cv2.imdecode(compressed_image, cv2.IMREAD_COLOR)
            if rgb is None:
                return None
            # 转换BGR到RGB
            rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
            return rgb
        except Exception as e:
            return None
    
    def extract_images_from_h5(self, h5_path: Path) -> Dict[str, List[np.ndarray]]:
        """
        从单个HDF5文件中提取所有RGB图像
        
        Args:
            h5_path: HDF5文件路径
            
        Returns:
            相机名称到图像列表的映射
        """
        images_by_camera = {}
        
        try:
            with h5py.File(h5_path, 'r') as f:
                # 检查是否有压缩标志
                is_compress = f.attrs.get('compress', True)
                
                # 检查RGB图像路径
                if 'observations' not in f or 'rgb_images' not in f['observations']:
                    return images_by_camera
                
                rgb_images_group = f['observations']['rgb_images']
                
                # 遍历所有相机
                for camera_name in rgb_images_group.keys():
                    camera_data = rgb_images_group[camera_name]
                    images = []
                    
                    # 提取该相机的所有帧
                    for frame_idx in range(len(camera_data)):
                        try:
                            if is_compress:
                                # 解码压缩图像
                                compressed_img = camera_data[frame_idx]
                                img = self.decode_image(compressed_img)
                                if img is not None:
                                    images.append(img)
                            else:
                                # 直接读取未压缩图像
                                images.append(camera_data[frame_idx])
                        except Exception as e:
                            continue
                    
                    if images:
                        images_by_camera[camera_name] = images
                        
        except Exception as e:
            self.errors.append(f"读取 {h5_path}: {str(e)}")
            
        return images_by_camera
    
    def save_images_by_camera(self, 
                             images_by_camera: Dict[str, List[np.ndarray]], 
                             relative_path: Path):
        """
        按相机分类保存图像
        
        Args:
            images_by_camera: 相机名称到图像列表的映射
            relative_path: 相对路径（embodiment/task/episode）
        """
        for camera_name, images in images_by_camera.items():
            # 构建输出目录: output_base/camera_name/embodiment/task/episode
            camera_output_dir = self.output_base_path / camera_name / relative_path
            camera_output_dir.mkdir(parents=True, exist_ok=True)
            
            # 保存所有帧
            for idx, img in enumerate(images):
                # 文件名格式: frame_0000.jpg, frame_0001.jpg, ...
                frame_name = f"frame_{idx:04d}.jpg"
                output_path = camera_output_dir / frame_name
                
                # 转换RGB回BGR用于OpenCV保存
                img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                cv2.imwrite(str(output_path), img_bgr)
                
                # 更新统计
                self.stats['total_images'] += 1
                if camera_name not in self.stats['cameras']:
                    self.stats['cameras'][camera_name] = 0
                self.stats['cameras'][camera_name] += 1
    
    def find_all_h5_files(self) -> List[tuple]:
        """
        查找所有HDF5文件
        
        Returns:
            (h5文件路径, 相对路径)的列表
        """
        h5_files = []
        
        print(f"🔍 扫描目录: {self.base_data_path}")
        
        # 遍历所有embodiment目录
        for embodiment_dir in self.base_data_path.iterdir():
            if not embodiment_dir.is_dir():
                continue
                
            embodiment_name = embodiment_dir.name
            
            # 遍历所有task目录
            for task_dir in embodiment_dir.iterdir():
                if not task_dir.is_dir():
                    continue
                    
                task_name = task_dir.name
                
                # 查找success_episodes/train下的所有episode
                train_dir = task_dir / 'success_episodes' / 'train'
                if not train_dir.exists():
                    continue
                
                for episode_dir in train_dir.iterdir():
                    if not episode_dir.is_dir():
                        continue
                        
                    episode_name = episode_dir.name
                    
                    # 查找data目录下的HDF5文件
                    data_dir = episode_dir / 'data'
                    if not data_dir.exists():
                        continue
                    
                    # 查找所有.hdf5文件
                    for h5_file in data_dir.glob('*.hdf5'):
                        # 构建相对路径
                        relative_path = Path(embodiment_name) / task_name / episode_name
                        h5_files.append((h5_file, relative_path))
        
        return h5_files
    
    def extract_all(self, limit: Optional[int] = None):
        """
        提取所有RGB图像
        
        Args:
            limit: 限制处理的HDF5文件数量（用于测试）
        """
        # 查找所有HDF5文件
        h5_files = self.find_all_h5_files()
        
        if not h5_files:
            print("❌ 未找到任何HDF5文件")
            return
        
        print(f"📊 找到 {len(h5_files)} 个HDF5文件\n")
        
        # 限制处理数量（如果指定）
        if limit:
            h5_files = h5_files[:limit]
            print(f"⚠️  限制处理前 {limit} 个文件\n")
        
        self.stats['total_h5_files'] = len(h5_files)
        
        # 使用tqdm处理每个HDF5文件
        with tqdm(total=len(h5_files), desc="提取图像", unit="文件") as pbar:
            for h5_path, relative_path in h5_files:
                try:
                    # 提取图像
                    images_by_camera = self.extract_images_from_h5(h5_path)
                    
                    if images_by_camera:
                        # 保存图像
                        self.save_images_by_camera(images_by_camera, relative_path)
                        self.stats['successful_h5_files'] += 1
                    else:
                        self.stats['failed_h5_files'] += 1
                        self.errors.append(f"无图像: {relative_path}")
                    
                    # 更新进度条
                    pbar.set_postfix({
                        '成功': self.stats['successful_h5_files'],
                        '失败': self.stats['failed_h5_files'],
                        '图像': self.stats['total_images']
                    })
                    
                except Exception as e:
                    self.stats['failed_h5_files'] += 1
                    self.errors.append(f"{relative_path}: {str(e)}")
                    pbar.set_postfix({
                        '成功': self.stats['successful_h5_files'],
                        '失败': self.stats['failed_h5_files']
                    })
                
                pbar.update(1)
        
        # 输出统计信息
        self.print_summary()
    
    def print_summary(self):
        """打印提取统计信息"""
        print(f"\n{'='*80}")
        print(f"✅ 提取完成！")
        print(f"{'='*80}")
        print(f"📊 统计信息:")
        print(f"   总HDF5文件数: {self.stats['total_h5_files']}")
        print(f"   成功处理: {self.stats['successful_h5_files']}")
        print(f"   处理失败: {self.stats['failed_h5_files']}")
        print(f"   提取图像总数: {self.stats['total_images']}")
        
        if self.stats['cameras']:
            print(f"\n📷 按相机统计:")
            for camera, count in sorted(self.stats['cameras'].items()):
                print(f"   {camera}: {count} 张")
        
        print(f"\n💾 输出目录: {self.output_base_path}")
        
        # 显示目录结构示例
        print(f"\n📁 输出目录结构:")
        print(f"   {self.output_base_path}/")
        if self.stats['cameras']:
            for camera in sorted(self.stats['cameras'].keys()):
                print(f"   ├── {camera}/")
                print(f"   │   ├── embodiment_name/")
                print(f"   │   │   ├── task_name/")
                print(f"   │   │   │   ├── episode_name/")
                print(f"   │   │   │   │   ├── frame_0000.jpg")
                print(f"   │   │   │   │   ├── frame_0001.jpg")
                print(f"   │   │   │   │   └── ...")
                break  # 只显示一个示例
        
        # 显示错误信息
        if self.errors:
            print(f"\n⚠️  发现 {len(self.errors)} 个错误")
            response = input("是否查看错误详情？(y/n): ")
            if response.lower() == 'y':
                print("\n错误列表:")
                for i, error in enumerate(self.errors[:50], 1):  # 最多显示50个
                    print(f"  {i}. {error}")
                if len(self.errors) > 50:
                    print(f"  ... 还有 {len(self.errors) - 50} 个错误未显示")


def main():
    parser = argparse.ArgumentParser(description='从HDF5文件批量提取RGB图像')
    parser.add_argument('--base_path', type=str, 
                       default='/home/vcj9002/jianshu/chengxuan/Data/robomind/data/benchmark1_0_compressed',
                       help='HDF5数据的基础路径')
    parser.add_argument('--output_path', type=str,
                       default='/home/vcj9002/jianshu/chengxuan/Data/robomind/data/3rgb',
                       help='输出图像的基础路径')
    parser.add_argument('--limit', type=int, default=None,
                       help='限制处理的文件数量（用于测试）')
    
    # 使用parse_known_args()以支持Jupyter notebook环境
    args, unknown = parser.parse_known_args()
    
    # 创建提取器
    extractor = RGBImageExtractor(
        base_data_path=args.base_path,
        output_base_path=args.output_path
    )
    
    print("="*80)
    print("🚀 RGB图像批量提取工具")
    print("="*80)
    print(f"📂 输入目录: {args.base_path}")
    print(f"💾 输出目录: {args.output_path}")
    if args.limit:
        print(f"⚠️  测试模式: 只处理前 {args.limit} 个文件")
    print("="*80 + "\n")
    
    # 执行提取
    extractor.extract_all(limit=args.limit)


if __name__ == "__main__":
    main()

🚀 RGB图像批量提取工具
📂 输入目录: /home/vcj9002/jianshu/chengxuan/Data/robomind/data/benchmark1_0_compressed
💾 输出目录: /home/vcj9002/jianshu/chengxuan/Data/robomind/data/3rgb

🔍 扫描目录: /home/vcj9002/jianshu/chengxuan/Data/robomind/data/benchmark1_0_compressed
📊 找到 12460 个HDF5文件



提取图像:   1%|          | 135/12460 [03:51<5:52:39,  1.72s/文件, 成功=135, 失败=0, 图像=50139]


KeyboardInterrupt: 

In [1]:
#!/usr/bin/env python3
"""
RGB图像批量提取工具 - 多线程版本
支持高并发（推荐128线程）
"""

import h5py
import numpy as np
import cv2
from pathlib import Path
from typing import Dict, List, Optional
from tqdm import tqdm
import argparse
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed


class RGBImageExtractor:
    """从HDF5批量提取RGB图像（多线程）"""
    
    def __init__(self, base_data_path: str, output_base_path: str):
        self.base_data_path = Path(base_data_path)
        self.output_base_path = Path(output_base_path)
        self.stats = {
            'total_h5_files': 0,
            'successful_h5_files': 0,
            'failed_h5_files': 0,
            'total_images': 0,
            'cameras': {}
        }
        self.errors = []
        self.stats_lock = threading.Lock()
        self.errors_lock = threading.Lock()
        
    def decode_image(self, compressed_image: np.ndarray) -> Optional[np.ndarray]:
        """解码压缩图像"""
        if compressed_image is None or len(compressed_image) == 0:
            return None
        try:
            rgb = cv2.imdecode(compressed_image, cv2.IMREAD_COLOR)
            if rgb is None:
                return None
            return cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        except:
            return None
    
    def extract_images_from_h5(self, h5_path: Path) -> Dict[str, List[np.ndarray]]:
        """从HDF5提取所有相机图像"""
        images_by_camera = {}
        try:
            with h5py.File(h5_path, 'r') as f:
                is_compress = f.attrs.get('compress', True)
                if 'observations' not in f or 'rgb_images' not in f['observations']:
                    return images_by_camera
                
                rgb_images_group = f['observations']['rgb_images']
                for camera_name in rgb_images_group.keys():
                    camera_data = rgb_images_group[camera_name]
                    images = []
                    for frame_idx in range(len(camera_data)):
                        try:
                            if is_compress:
                                img = self.decode_image(camera_data[frame_idx])
                                if img is not None:
                                    images.append(img)
                            else:
                                images.append(camera_data[frame_idx])
                        except:
                            continue
                    if images:
                        images_by_camera[camera_name] = images
        except Exception as e:
            self.errors.append(f"读取失败 {h5_path}: {str(e)}")
        return images_by_camera
    
    def save_images_by_camera(self, images_by_camera: Dict[str, List[np.ndarray]], relative_path: Path):
        """按相机保存图像（线程安全）"""
        local_stats = {'total_images': 0, 'cameras': {}}
        
        for camera_name, images in images_by_camera.items():
            camera_output_dir = self.output_base_path / camera_name / relative_path
            camera_output_dir.mkdir(parents=True, exist_ok=True)
            
            for idx, img in enumerate(images):
                frame_name = f"frame_{idx:04d}.jpg"
                output_path = camera_output_dir / frame_name
                img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                cv2.imwrite(str(output_path), img_bgr)
                
                local_stats['total_images'] += 1
                if camera_name not in local_stats['cameras']:
                    local_stats['cameras'][camera_name] = 0
                local_stats['cameras'][camera_name] += 1
        
        with self.stats_lock:
            self.stats['total_images'] += local_stats['total_images']
            for camera, count in local_stats['cameras'].items():
                if camera not in self.stats['cameras']:
                    self.stats['cameras'][camera] = 0
                self.stats['cameras'][camera] += count
    
    def find_all_h5_files(self) -> List[tuple]:
        """查找所有HDF5文件"""
        h5_files = []
        print(f"🔍 扫描目录: {self.base_data_path}")
        
        for embodiment_dir in self.base_data_path.iterdir():
            if not embodiment_dir.is_dir():
                continue
            for task_dir in embodiment_dir.iterdir():
                if not task_dir.is_dir():
                    continue
                train_dir = task_dir / 'success_episodes' / 'train'
                if not train_dir.exists():
                    continue
                for episode_dir in train_dir.iterdir():
                    if not episode_dir.is_dir():
                        continue
                    data_dir = episode_dir / 'data'
                    if not data_dir.exists():
                        continue
                    for h5_file in data_dir.glob('*.hdf5'):
                        relative_path = Path(embodiment_dir.name) / task_dir.name / episode_dir.name
                        h5_files.append((h5_file, relative_path))
        return h5_files
    
    def process_single_h5(self, h5_path: Path, relative_path: Path) -> Dict:
        """处理单个HDF5（用于多线程）"""
        result = {'success': False, 'error': None}
        try:
            images_by_camera = self.extract_images_from_h5(h5_path)
            if images_by_camera:
                self.save_images_by_camera(images_by_camera, relative_path)
                result['success'] = True
            else:
                result['error'] = f"无图像: {relative_path}"
        except Exception as e:
            result['error'] = f"{relative_path}: {str(e)}"
        return result
    
    def extract_all(self, limit: Optional[int] = None, num_threads: int = 128):
        """多线程提取所有图像"""
        h5_files = self.find_all_h5_files()
        
        if not h5_files:
            print("❌ 未找到任何HDF5文件")
            return
        
        print(f"📊 找到 {len(h5_files)} 个HDF5文件")
        print(f"🔧 使用 {num_threads} 个线程并行处理\n")
        
        if limit:
            h5_files = h5_files[:limit]
            print(f"⚠️  限制处理前 {limit} 个文件\n")
        
        self.stats['total_h5_files'] = len(h5_files)
        
        with ThreadPoolExecutor(max_workers=num_threads) as executor:
            future_to_file = {
                executor.submit(self.process_single_h5, h5_path, relative_path): (h5_path, relative_path)
                for h5_path, relative_path in h5_files
            }
            
            with tqdm(total=len(h5_files), desc="提取图像", unit="文件") as pbar:
                for future in as_completed(future_to_file):
                    result = future.result()
                    
                    with self.stats_lock:
                        if result['success']:
                            self.stats['successful_h5_files'] += 1
                        else:
                            self.stats['failed_h5_files'] += 1
                        pbar.set_postfix({
                            '成功': self.stats['successful_h5_files'],
                            '失败': self.stats['failed_h5_files'],
                            '图像': self.stats['total_images']
                        })
                    
                    if result['error']:
                        with self.errors_lock:
                            self.errors.append(result['error'])
                    pbar.update(1)
        
        self.print_summary()
    
    def print_summary(self):
        """输出统计信息"""
        print(f"\n{'='*80}")
        print(f"✅ 提取完成！")
        print(f"{'='*80}")
        print(f"📊 统计:")
        print(f"   总文件: {self.stats['total_h5_files']}")
        print(f"   成功: {self.stats['successful_h5_files']}")
        print(f"   失败: {self.stats['failed_h5_files']}")
        print(f"   图像数: {self.stats['total_images']}")
        
        if self.stats['cameras']:
            print(f"\n📷 按相机统计:")
            for camera, count in sorted(self.stats['cameras'].items()):
                print(f"   {camera}: {count} 张")
        
        print(f"\n💾 输出: {self.output_base_path}")
        
        if self.errors and len(self.errors) <= 20:
            print(f"\n⚠️  错误 ({len(self.errors)}):")
            for error in self.errors[:20]:
                print(f"   - {error}")


def main():
    parser = argparse.ArgumentParser(description='RGB图像批量提取（多线程）')
    parser.add_argument('--base_path', type=str, 
                       default='/home/vcj9002/jianshu/chengxuan/Data/robomind/data/benchmark1_0_compressed',
                       help='输入路径')
    parser.add_argument('--output_path', type=str,
                       default='/home/vcj9002/jianshu/chengxuan/Data/robomind/data/3rgb',
                       help='输出路径')
    parser.add_argument('--limit', type=int, default=None, help='限制文件数（测试用）')
    parser.add_argument('--threads', type=int, default=128, help='线程数（默认128）')
    
    args, _ = parser.parse_known_args()
    
    extractor = RGBImageExtractor(
        base_data_path=args.base_path,
        output_base_path=args.output_path
    )
    
    print("="*80)
    print("🚀 RGB图像批量提取（多线程版）")
    print("="*80)
    print(f"📂 输入: {args.base_path}")
    print(f"💾 输出: {args.output_path}")
    print(f"🧵 线程: {args.threads}")
    if args.limit:
        print(f"⚠️  测试: 前 {args.limit} 个文件")
    print("="*80 + "\n")
    
    extractor.extract_all(limit=args.limit, num_threads=args.threads)


if __name__ == "__main__":
    main()

🚀 RGB图像批量提取（多线程版）
📂 输入: /home/vcj9002/jianshu/chengxuan/Data/robomind/data/benchmark1_0_compressed
💾 输出: /home/vcj9002/jianshu/chengxuan/Data/robomind/data/3rgb
🧵 线程: 128

🔍 扫描目录: /home/vcj9002/jianshu/chengxuan/Data/robomind/data/benchmark1_0_compressed
📊 找到 12460 个HDF5文件
🔧 使用 128 个线程并行处理



提取图像:   6%|▌         | 724/12460 [08:50<6:32:59,  2.01s/文件, 成功=624, 失败=100, 图像=274677][ERROR:169@537.842] global loadsave.cpp:1324 imdecode_ imdecode_(''): can't read header: OpenCV(4.12.0) /io/opencv/modules/imgcodecs/src/grfmt_bmp.cpp:99: error: (-215:Assertion failed) size > 0 in function 'readHeader'

提取图像:  71%|███████   | 8870/12460 [1:51:48<56:29,  1.06文件/s, 成功=7373, 失败=1497, 图像=3074079]  [ERROR:0@6709.028] global loadsave.cpp:1324 imdecode_ imdecode_(''): can't read header: OpenCV(4.12.0) /io/opencv/modules/imgcodecs/src/grfmt_bmp.cpp:108: error: (-215:Assertion failed) m_rle_code_ >= 0 && m_rle_code_ <= BMP_BITFIELDS in function 'readHeader'

提取图像:  72%|███████▏  | 8915/12460 [1:52:50<50:56,  1.16文件/s, 成功=7388, 失败=1527, 图像=3080628]  [ERROR:111@6771.400] global loadsave.cpp:1324 imdecode_ imdecode_(''): can't read header: OpenCV(4.12.0) /io/opencv/modules/imgcodecs/src/grfmt_bmp.cpp:108: error: (-215:Assertion failed) m_rle_code_ >= 0 && m_rle_code_ <= BMP_BITFIELDS in function


✅ 提取完成！
📊 统计:
   总文件: 12460
   成功: 10285
   失败: 2175
   图像数: 4398978

📷 按相机统计:
   camera_left: 1466326 张
   camera_right: 1466326 张
   camera_top: 1466326 张

💾 输出: /home/vcj9002/jianshu/chengxuan/Data/robomind/data/3rgb
